In [1]:
import sys

In [2]:
!git clone https://github.com/bstabler/TransportationNetworks.git

Cloning into 'TransportationNetworks'...
remote: Enumerating objects: 554, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 554 (delta 9), reused 18 (delta 3), pack-reused 517
Receiving objects: 100% (554/554), 57.44 MiB | 14.26 MiB/s, done.
Resolving deltas: 100% (283/283), done.
Updating files: 100% (135/135), done.


In [3]:
!pip install --upgrade scipy==1.8.0 networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 MB 7.6 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1


In [4]:
import pandas as pd
import numpy as np
import networkx as nx

network = pd.read_csv(r"/content/TransportationNetworks/Anaheim/Anaheim_net.tntp",skiprows=8, sep='\t', header=0)
network.reset_index(drop=True, inplace=True)

In [5]:
network_2 = pd.read_csv(r"/content/TransportationNetworks/Anaheim/Anaheim_net.tntp",nrows=8, header=None)
network_2.reset_index(drop=True, inplace=True)
s= network_2.loc[1]
s=str(s)
s=s.split(">")
s=s[1]
if "\\t" in s:
  s=s.split("\\t")
if "\nName" in s:
  s=s.split("\n")
while('' in s):
    s.remove('')
no_of_nodes=int(s[0])

metadata for first through node

In [6]:
s= network_2.loc[2]
s=str(s)
s=s.split(">")
s=s[1]
if "\\t" in s:
  s=s.split("\\t")
if "\nName" in s:
  s=s.split("\n")
while('' in s):
    s.remove('')
first_through_node=int(s[0])

In [ ]:

class Arc:
    def __init__(self, tail_node: int, head_node: int, capacity: float,
                 length: float, free_flow_time: float, b: float,
                 power: float, speed_limit: float, toll: float, type_link: int):
        self.tail_node = tail_node
        self.head_node = head_node
        self.capacity = capacity
        self.length = length
        self.free_flow_time = free_flow_time
        self.b = b
        self.power = power
        self.speed_limit = speed_limit
        self.toll = toll
        self.type_link = type_link
        self.flow = 0
        self.time = 

In [ ]:
arcs = dict()
for index,rows in network.iterrows():
  arcs[index+1] = Arc(int(rows.init_node), int(rows.term_node), float(rows.capacity)
                        , float(rows.length), float(rows.free_flow_time), float(rows.b)
                        , float(rows.power), float(rows.speed), float(rows.toll)
                        , int(rows.link_type))

In [ ]:
class Node:
  def __init__(self, nodeNumber):
    self.nodeNumber = nodeNumber
    self.adjNodes = []
    self.adjArcs = []
    self.dist = sys.maxsize
    self.parNode = -1
    self.predArc = 0

  def addAdjNode(self, node):
    self.adjNodes.append(node)

  def addAdjArcs(self, n):
    self.adjArcs.append(n)

In [ ]:
nodes = dict()
for key in arcs:
  if not arcs[key].tail_node in nodes:
    nodes[arcs[key].tail_node] = Node(arcs[key].tail_node) 
  nodes[arcs[key].tail_node].addAdjNode(arcs[key].head_node)
  nodes[arcs[key].tail_node].addAdjArcs(key)


  


Label correcting Method

In [ ]:
import time
def label_corr_mthd(origin):
  sel = []
  for key in nodes:
    nodes[key].dist = sys.maxsize
    nodes[key].parNode = -1
  nodes[origin].dist = 0
  nodes[origin].parNode = origin
  sel.append(origin)


  
  while len(sel) !=0:

    i = sel[0]
    # print("i=", i)
    sel.remove(i)
    # print(sel)
    for j in nodes[i].adjArcs:
      # print("j=",j)
      if nodes[arcs[j].head_node].dist > nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time:
       nodes[arcs[j].head_node].dist = nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time
      #  print(arcs[j].head_node,nodes[arcs[j].head_node].dist)
       nodes[arcs[j].head_node].parNode = i
       nodes[arcs[j].head_node].predArc = j
       if not arcs[j].head_node in sel:
        #  print("jo dalne wale hai")
         sel.append(arcs[j].head_node)
        #  print("sel",sel)
  for key in nodes:
    pred_arc = nodes[key].predArc
    li = [nodes[key].nodeNumber]
    # print("shortest path for node ",key," is ")
    while key != origin:
      li.append(nodes[key].parNode)
      key = nodes[key].parNode
    li.reverse()
    # print(li)

start = time.time()
for key in nodes:
  # print("for ", key," as origin:")
  label_corr_mthd(key)
  # print("------------")
  # print()
  # print()
end = time.time()
# print("time taken for dijkstra's algorithm is:")
print((end-start)/no_of_nodes)

0.00660513284114691


dijkstra's algorithm

In [ ]:
#normal dijkstra
def minarg(s_bar):
  minValue = min([nodes[i].dist for i in s_bar])
  for key in nodes:
    if (minValue == nodes[key].dist and key in s_bar):
      # print(key)
      return(key)
      
def dijkstra(origin):
  
  s=[]
  s_bar = []
  for key in nodes:
    nodes[key].dist = sys.maxsize
    nodes[key].parNode = -1
    s_bar.append(key)
  nodes[origin].dist = 0
  nodes[origin].parNode = origin

  
  while len(s_bar) !=0:
   i = minarg(s_bar)
   s.append(i)
   s_bar.remove(i)
   for j in nodes[i].adjArcs:
    #  nodes[j].predArc = nodes[i].adjArcs[j]
     if nodes[arcs[j].head_node].dist > nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time:
       nodes[arcs[j].head_node].dist = nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time
       nodes[arcs[j].head_node].parNode = i
    
  for key in nodes:
      pred_arc = nodes[key].predArc
      li = [nodes[key].nodeNumber]
      # print("shortest path for node ",key," is ")
      while key != origin:
        li.append(nodes[key].parNode)
        key = nodes[key].parNode
      li.reverse()
      # print(li)


start = time.time()
for key in nodes:
  # print("for ", key," as origin:")
  dijkstra(key)
  # print("------------")
  # print()
  # print()
end = time.time()
# print("time taken for dijkstra's algorithm is:")
print((end-start)/no_of_nodes)
      


0.02431868016719818


modified label correcting method


In [ ]:


def mod_label_corr_mthd(origin):
  sel = []
  for key in nodes:
    nodes[key].dist = sys.maxsize
    nodes[key].parNode = -1
  nodes[origin].dist = 0
  nodes[origin].parNode = origin
  sel.append(origin)


  
  while len(sel) !=0:
    i = sel[0]
    # print("i=", i)
    sel.remove(i)
    # print(sel)
    for j in nodes[i].adjArcs:
      # print("j=",j)
      if arcs[j].head_node<first_through_node:
        continue
      if nodes[arcs[j].head_node].dist > nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time:
       nodes[arcs[j].head_node].dist = nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time
      #  print(arcs[j].head_node,nodes[arcs[j].head_node].dist)
       nodes[arcs[j].head_node].parNode = i
       if not arcs[j].head_node in sel:
        #  print("jo dalne wale hai")
         sel.append(arcs[j].head_node)
        #  print("sel",sel)
  for key in nodes:
    if nodes[key].dist == sys.maxsize:
      print("no direct path exist for node", key )
      continue
    pred_arc = nodes[key].predArc
    li = [nodes[key].nodeNumber]
    print("shortest path for node ",key," is ")
    while key != origin:
      li.append(nodes[key].parNode)
      key = nodes[key].parNode
    li.reverse()
    print(li)
for key in nodes:
  print("for ", key," as origin:")
  mod_label_corr_mthd(key)
  print("------------")
  print()
  print()

modified dijkstra's algorithm

In [ ]:

#modified minarg
def mod_minarg(s_bar):
  minValue = sys.maxsize
  key_1=-1
  for key in nodes:
    if ((minValue > nodes[key].dist) and key in s_bar):
      # print(nodes[key].dist)
      minValue = nodes[key].dist
      key_1 = key
      # print(key_1)

  return(key_1)

# modified dijkstra
def mod_dijkstra(origin):
  
  s=[]
  s_bar = []
  if origin < first_through_node:
    s_bar = [origin]
  for key in nodes:
    nodes[key].dist = sys.maxsize
    nodes[key].parNode = -1
    if key >= first_through_node:
      s_bar.append(key)
  nodes[origin].dist = 0
  nodes[origin].parNode = origin

  
  while len(s_bar) !=0:
   i = mod_minarg(s_bar)
   if i == -1 :
     break
   s.append(i)
   s_bar.remove(i)
   for j in nodes[i].adjArcs:
     if nodes[arcs[j].head_node].dist > nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time:
       
       nodes[arcs[j].head_node].dist = nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time
      #  print(nodes[arcs[j].head_node].dist)
       nodes[arcs[j].head_node].parNode = i
  for key in nodes:
    if nodes[key].dist == sys.maxsize:
      print("no direct path exist for node", key )
      continue
    pred_arc = nodes[key].predArc
    li = [nodes[key].nodeNumber]
    print("shortest path for node ",key," is ")
    while key != origin:
      li.append(nodes[key].parNode)
      key = nodes[key].parNode
    li.reverse()
    print(li)

     
for key in nodes:
  print("for ", key," as origin:")
  mod_dijkstra(key)
  print("------------")
  print()
  print()

Results on NetworkX

In [ ]:
import networkx as nx

In [ ]:
start = time.time()
g = nx.from_pandas_edgelist(network,'init_node','term_node',create_using=nx.MultiDiGraph(),edge_attr="free_flow_time")
li=nx.shortest_path(g, source=None, target=None, weight="free_flow_time", method='dijkstra')
end = time.time()
print("time taken for dijkstra's algorithm is:")
print((end-start)/no_of_nodes)


time taken for dijkstra's algorithm is:
0.002492742469677558


comparison of results of label correcting method with networkx 

In [ ]:
def len_label_corr_mthd(origin,length):
  sel = []
  for key in nodes:
    nodes[key].dist = sys.maxsize
    nodes[key].parNode = -1
  nodes[origin].dist = 0
  nodes[origin].parNode = origin
  sel.append(origin)


  
  while len(sel) !=0:

    i = sel[0]
    sel.remove(i)
    for j in nodes[i].adjArcs:
      if nodes[arcs[j].head_node].dist > nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time:
       nodes[arcs[j].head_node].dist = nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time
       nodes[arcs[j].head_node].parNode = i
       nodes[arcs[j].head_node].predArc = j
       if not arcs[j].head_node in sel:
         sel.append(arcs[j].head_node)

  for key in nodes:
    temp = key
    pred_arc = nodes[key].predArc
    dist = 0.0
    while key != origin:
      dist  += arcs[pred_arc].free_flow_time
      key = nodes[key].parNode
      pred_arc = nodes[key].predArc
    length[temp] = dist

In [ ]:
for key in nodes:
  length = dict()
  len_label_corr_mthd(key,length)
  length_nx = nx.shortest_path_length(g, source=key, target=None, weight="free_flow_time", method='dijkstra')
  for l in length:
    if format(length_nx[l],".2f") != format(length[l],".2f"):
      print("Lengths are not equal for ", l ,"when",key," is the origin" )
    else:
      print("Lengths are equal for ", l ,"when", key," is the origin")
    

comparison of results of label setting method with networkx 

In [ ]:
#normal dijkstra
def minarg(s_bar):
  minValue = min([nodes[i].dist for i in s_bar])
  for key in nodes:
    if (minValue == nodes[key].dist and key in s_bar):
      # print(key)
      return(key)
      
def len_dijkstra(origin,length):
  
  s=[]
  s_bar = []
  for key in nodes:
    nodes[key].dist = sys.maxsize
    nodes[key].parNode = -1
    s_bar.append(key)
  nodes[origin].dist = 0
  nodes[origin].parNode = origin

  
  while len(s_bar) !=0:
   i = minarg(s_bar)
   s.append(i)
   s_bar.remove(i)
   for j in nodes[i].adjArcs:
    #  nodes[j].predArc = nodes[i].adjArcs[j]
     if nodes[arcs[j].head_node].dist > nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time:
       nodes[arcs[j].head_node].dist = nodes[arcs[j].tail_node].dist + arcs[j].free_flow_time
       nodes[arcs[j].head_node].parNode = i
       nodes[arcs[j].head_node].predArc = j
    
  for key in nodes:
    temp = key
    pred_arc = nodes[key].predArc
    dist = 0.0
    while key != origin:
      dist  += arcs[pred_arc].free_flow_time
      key = nodes[key].parNode
      pred_arc = nodes[key].predArc
    length[temp] = dist



In [ ]:
for key in nodes:
  length = dict()
  len_dijkstra(key,length)
  length_nx = nx.shortest_path_length(g, source=key, target=None, weight="free_flow_time", method='dijkstra')
  for l in length:
    if format(length_nx[l],".2f") != format(length[l],".2f"):
      print("Lengths are not equal for ", l ,"when",key," is the origin" )
    else:
      print("Lengths are equal for ", l ,"when", key," is the origin")

3rd question

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx

filename = "Chicago-Sketch/ChicagoSketch_trips.tntp"
df = pd.read_csv(f"/content/TransportationNetworks/{filename}", skiprows=3, on_bad_lines='skip', header=None)


# "SiouxFalls/SiouxFalls_trips.tntp"
# "Chicago-Sketch/ChicagoSketch_trips.tntp"
# "Eastern-Massachusetts/EMA_trips.tntp"
# "Anaheim/Anaheim_trips.tntp"

In [ ]:
num_rows = pd.read_csv(f"/content/TransportationNetworks/{filename}", nrows=2, on_bad_lines='skip', header=None)

In [ ]:
num_rows

In [ ]:
num_of_zones = num_rows[0][0].split(">")
flow = num_rows[0][1].split(">")
flow = float(flow[1])
num_of_zones = int(num_of_zones[1])

In [ ]:
data = [row.split("\t") for row in df[0]]

In [ ]:
OD_matrix = [[0.0 for j in range(num_of_zones)] for i in range(num_of_zones)]
i = -1  # initialize i to -1
for rows in data:
    if rows[0].startswith('Origin'):
        i += 1 # increment i for each new origin
    else:
        l = rows[0].split(';')
        for values in l:
            v = values.split(":")
            try:
                j = int(v[0])-1
                OD_matrix[i][j] = float(v[1])
            except:
                IndexError


In [ ]:
OD_matrix

In [ ]:
df[0]

In [ ]:
OD_matrix = [[0 for j in range(24)] for i in range(24)]
for rows in data:
  if len(rows) == 2:
    # print(rows[1])
    i = int(rows[1])-1
  else:
    # print(rows)
    l=rows[0].split(';')
    for values in l:
      v=values.split(":")
      try:
        print(i)
        print(v[0],v[1])
        j = int(v[0])-1
        OD_matrix[i][j] = float(v[1])
      except:
        IndexError

